In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm, binom
from scipy import stats

In [2]:
from src.MM_estimation import calc_variance_of_default_rate
from src.MM_estimation import estimate_w_factor_loading
from src.ML_estimation import calculate_my_likelihood
from src.ML_estimation import calculate_my_likelihood_arr
from src.ML_estimation import maximum_likelihood_estimation
from src.monte_carlo import monte_carlo_MLE, expected_value_of_function_monte_carlo, likelihood_mc
from src.sucess_probability import p_g

In [3]:
df = pd.read_csv('SP_historical_PD_data.csv', sep=';')
df["pd_total"] = df["Default rate (%)"] / 100
df["pd_inv"] = df["Investment-grade default rate (%)"] / 100
df["pd_spec"] = df["Speculative-grade default rate (%)"] / 100
# Calculate the number of obligors
df['num_of_inv_grades'] = (df['Investment-grade defaults'] / (df["pd_inv"])).round()
df['num_of_spec_grades'] = (
            df['Speculative-grade defaults'] / (df["pd_spec"])).round().astype(int)
df['num_of_total_grades'] = (df['Total defaults*'] / (df["pd_total"])).round().astype(int)

# Fill-out the missing values in num_of_inv_grades column with the difference between num_of_total_grades and num_of_spec_grades
df['num_of_inv_grades'] = np.where(df['num_of_inv_grades'].isna(), df['num_of_total_grades'] - df['num_of_spec_grades'],
                                   df['num_of_inv_grades']).astype(int)

In [4]:
# MM for speculative grade
MM_params = estimate_w_factor_loading(df["pd_spec"], df["num_of_spec_grades"], initial_guess=0.27)
print("Estimated parameters for speculative grade: ", MM_params[0], norm.ppf(MM_params[1]))

Estimated parameters for speculative grade:  0.07602889962521624 -1.7498743361043698


In [5]:
# ML for speculative grade
ML_params = maximum_likelihood_estimation(df["Speculative-grade defaults"], df["num_of_spec_grades"], norm.pdf, p_g, 0.27, -2, [(0, 1), (-np.inf, np.inf)])
print("Estimated parameters for speculative grade: ", ML_params)

Estimated parameters for speculative grade:  (0.0, -1.78533136062324)


In [6]:
Monte_carlo_params = monte_carlo_MLE([df["Speculative-grade defaults"].sum()], [df["num_of_spec_grades"].sum()], p_g, [0.27], [-2], [(0, 1), (-np.inf, np.inf)], num_of_simulations=10000)
print("Estimated parameters for speculative grade: ", Monte_carlo_params.x)

Estimated parameters for speculative grade:  [ 0.28771878 -1.72499994]


In [7]:
# An example for the calculation of the integrand
test_d_g_arr = np.array([5, 8])
test_n_g_arr = np.array([50, 100])
prob_dens_func = norm.pdf
test_w_g_arr = np.array([0.9, 0.9])
test_gamma_g_arr = np.array([-1.6449, -1.6449])

calculate_my_likelihood_arr(test_d_g_arr, test_n_g_arr, p_g, prob_dens_func, test_w_g_arr, test_gamma_g_arr)

0.0008893194009710714

In [8]:
test_lamda_func = lambda x: np.prod([stats.binom.pmf(d, n, p_g(x, w_g, gamma_g)) for d, n, w_g, gamma_g in zip(test_d_g_arr, test_n_g_arr, test_w_g_arr, test_gamma_g_arr)], axis = 0)

expected_value_of_function_monte_carlo(test_lamda_func, num_samples=1000000)

0.0008960398476656619

In [9]:
likelihood_mc(test_w_g_arr[0], test_gamma_g_arr, test_d_g_arr, test_n_g_arr, p_g, num_of_simulations=1000000)

0.0008843317041947533

In [10]:
#TO-DO fix this in likelihood_mc function:
"""
list(zip(test_d_g_arr, test_n_g_arr, test_w_g_arr, test_gamma_g_arr))
list(zip(test_d_g_arr, test_n_g_arr, test_w_g_arr, [-1.96]]))
"""

'\nlist(zip(test_d_g_arr, test_n_g_arr, test_w_g_arr, test_gamma_g_arr))\nlist(zip(test_d_g_arr, test_n_g_arr, test_w_g_arr, [-1.96]]))\n'

In [11]:
from src.sucess_probability import p_g

In [12]:
from src.data_generator import generate_default_buckets
from scipy.optimize import minimize

In [13]:
#np.random.seed(3123) # Fix the seed for reproducibility
np.random.seed()
time_horizon = 40
factor_loading_list = [0.45, 0.45, 0.45]
num_of_obligors_list = [3000, 3000, 3000]
gamma_list = [-2.9, -2.3, -1.6]
d_g_list = generate_default_buckets(factor_loading_list,num_of_obligors_list, gamma_list,time_horizon)
# multiply all num_of_obligors with time_horizon
n_g_list = [num_of_obligors_list[i] * time_horizon for i in range(len(num_of_obligors_list))]

In [14]:
d_g_list, n_g_list

([198, 1136, 6194], [120000, 120000, 120000])

In [15]:
#SP data
# d_g_list = np.array([64, 234, 1027, 5030, 29903])
# n_g_list = 40 * np.array([3000, 3000, 3000, 3000, 3000])
# 
# d_g_list, n_g_list

In [16]:
from scipy.integrate import quad

def calculate_my_likelihood_arr(d_g_arr, n_g_arr, p_g, prob_dens_func, w_g_arr, gamma_g_arr):
    """
    Numerically calculates the value of L(d_g_arr) for multiple grades based on the given formula.

    Parameters:
        d_g_arr (numpy.array(int)): Values of d_g's by grades
        n_g_arr (numpy.array(int)): Values of n_g's by grades
        p_g (callable): The p_g function representing the probability density function.
        prob_dens_func (callable): The pdf_g function representing the probability density function.
        w_g_arr (numpy.array(float)): Parameter 'w_g's by grades
        gamma_g_arr (numpy.array(float)): Parameter 'gamma_g's by grades.

    Returns:
        float: Numerical approximation of the integral.
    """

    integrand = lambda x: np.prod(binom.pmf(d_g_arr, n_g_arr, p_g(x, w_g_arr, gamma_g_arr))) * prob_dens_func(x)

    result, _ = quad(integrand, -3, 3) # , points=5, epsrel = 1e-012

    return result

In [17]:
def parameter_estimation(default_list, num_of_obligors_over_time, factor_loading_init, gamma_list_init):
    initial_guess = gamma_list_init + factor_loading_init
    
    num_of_gamma = len(gamma_list_init)
    num_of_factor_loading = len(factor_loading_init)
    # bound = num_of_gamma * (-5, 5) + num_of_factor_loading * (-1, 1)
    bounds = num_of_gamma * [(-5, 5)] + num_of_factor_loading * [(-1, 1)]
    # print(bounds)
    # print(initial_guess)
    # Optimization
    objective_function = lambda params: -np.log(calculate_my_likelihood_arr(
        default_list, num_of_obligors_over_time, p_g, norm.pdf, params[num_of_gamma], params[0:num_of_gamma]
    ))
    
    result = minimize(objective_function,
                  initial_guess,
                  method="Nelder-Mead",
                  bounds=bounds,
                  options={
                      'disp': True})
    
    return result

In [18]:
%%time
#factor_loading_init = [0.45]
factor_loading_init = [0.44]
# gamma_list_init = [-2.9, -2.3, -1.6]
gamma_list_init = [-2.87, -2.3, -1.6]
# gamma_list_init = [-2.05, -1.85, -1.6, -1.6, -1.6]
ML_params = parameter_estimation(d_g_list, n_g_list, factor_loading_init, gamma_list_init)
#print("Estimated parameters for speculative grade: ", ML_params)

Optimization terminated successfully.
         Current function value: 18.658806
         Iterations: 78
         Function evaluations: 182
CPU times: total: 5.47 s
Wall time: 5.6 s


In [19]:
ML_params.message

'Optimization terminated successfully.'

In [20]:
#real gamma: [-2.0, -1.8, -1.3]

In [21]:
ML_params.x

array([-2.82753284, -2.29998685, -1.67407578,  0.43998897])

In [22]:
-np.log(calculate_my_likelihood_arr(d_g_list, n_g_list, p_g, prob_dens_func, ML_params.x[3], ML_params.x[0:3]))

18.65880578771864

In [23]:
-np.log(likelihood_mc(ML_params.x[3], ML_params.x[0:3], d_g_list, n_g_list, p_g, num_of_simulations=1000000))

19.640462340924813

In [24]:
-np.log(likelihood_mc(factor_loading_init, gamma_list_init, d_g_list, n_g_list, p_g, num_of_simulations=1000000))

37.48633421546001

In [25]:
-np.log(likelihood_mc(factor_loading_list[0], gamma_list, d_g_list, n_g_list, p_g, num_of_simulations=1000000))

47.0720856257602

In [26]:
-np.log(calculate_my_likelihood_arr(d_g_list, n_g_list, p_g, prob_dens_func, factor_loading_list, gamma_list))

51.15128460268474

In [27]:
def w_calc_func(a, b):
    return -b/ np.sqrt(b**2 + 1)

def gamma_calc_func(a, b):
    return a * np.sqrt(1 - w_calc_func(a, b)**2)

def a_calc_func(w, gamma):
    return gamma / np.sqrt(1 - w**2)

def b_calc_func(w, gamma):
    return -w / np.sqrt(1 - w**2)

def calculate_variable_changed_likelihood_arr(d_g_arr, n_g_arr, p_g, prob_dens_func, a, b):
    integrand = lambda x: np.prod(binom.pmf(d_g_arr, n_g_arr, norm.cdf(a*x+b))) * prob_dens_func(x)

    result, _ = quad(integrand, -3, 3)

    return result

In [28]:
a = [a_calc_func(factor_loading_list[i], gamma_list[i]) for i in range(3)]
b = [b_calc_func(factor_loading_list[i], gamma_list[i]) for i in range(3)]
a, b

([-3.2473765635439547, -2.5755055503969295, -1.7916560350587338],
 [-0.5039032598602688, -0.5039032598602688, -0.5039032598602688])

In [29]:
%%time
# Different gamma and same factor loading parameter

# MLE condition and initial guess
initial_guess = np.array([-3.247, -2.576, -1.792, -0.50])
bounds = [(-5, 5), (-5, 5), (-5, 5), (-5, 5)]

# Function to be minimized in weight parameter
objective_function = lambda params: -np.log(calculate_variable_changed_likelihood_arr(d_g_list, n_g_list, p_g, norm.pdf, 
                                                                 np.repeat(params[3], 3), 
                                                                 params[0:3]))

result = minimize(objective_function,
                  initial_guess,
                  method="Nelder-Mead",
                  bounds=bounds,
                  options={
                      'disp': True})
# Method can be Nelder-Mead or Powell

# The optimal weight parameter
optimal_weight = result.x
print(f"The optimal weight parameter is {optimal_weight}")
print(result.message)

Optimization terminated successfully.
         Current function value: 18.658806
         Iterations: 80
         Function evaluations: 160
The optimal weight parameter is [-3.21409413 -2.57187844 -1.87304655 -0.50822791]
Optimization terminated successfully.
CPU times: total: 4.52 s
Wall time: 4.51 s


In [30]:
result.x

array([-3.21409413, -2.57187844, -1.87304655, -0.50822791])

In [31]:
b_result = result.x[3]
a_result = [result.x[i] for i in range(3)]
w_result = [w_calc_func(a_result[i], b_result) for i in range(3)]
gamma_result = [gamma_calc_func(a_result[i], b_result) for i in range(3)]

w_result, gamma_result

([0.4530719424852357, 0.4530719424852357, 0.4530719424852357],
 [-2.8652812090681987, -2.2927626474283787, -1.6697722189139166])

In [32]:
import random

In [33]:
def expected_value_of_function_monte_carlo(f, num_samples=100000):
    """
    Calculates the expected value of a given function f(x) where x follows the standard normal distribution.

    Parameters:
        f (callable): The function f(x) to calculate the expected value.
        num_samples (int): The number of samples to use in the Monte Carlo simulation.

    Returns:
        float: The estimated expected value of the function.
    """
    # Generate random samples from the standard normal distribution
    random_samples = np.random.normal(size=num_samples)

    # Calculate the expected value using the Monte Carlo simulation
    estimated_expected_value = np.mean(f(random_samples))

    return estimated_expected_value

In [34]:
def likelihood_mc(w_g, gamma_g, d_g, n_g, p_g, num_of_simulations):
    # likelihood function
    likelihood_function = lambda x: np.prod([stats.binom.pmf(d, n, p_g(x, w_g=w_g, gamma_g=gamma)) for d, n, gamma in zip(d_g, n_g, gamma_g)], axis=0)

    return expected_value_of_function_monte_carlo(likelihood_function, num_of_simulations)

In [35]:
# Monte Carlo
def monte_carlo_MLE(d_g, n_g, p_g, w_initial, gamma_initial, bounds, num_of_simulations, seed=None):
    """
    Estimate w_g and gamma_g using the maximum likelihood estimation method.
    Parameters:
        d_g (list): number of defaults.
        n_g (list): number of obligors.
        p_g (callable): The p_g function representing the probability density function.
        w_initial (float): Initial guess for w_g.
        gamma_initial (float): Initial guess for gamma_g.
        bounds (list): Bounds for the minimization algorithm.
        num_of_simulations (int): Number of simulations for the Monte Carlo method.
        seed (int): Seed for the random number generator.
    Returns:
        tuple: Estimated w_g and gamma_g.
    """
    if seed is None:
        seed = random.randint(1, 10000)

    np.random.seed(seed)

    # objective function
    objective_function = lambda params: -np.log(likelihood_mc(params[0], params[1:], d_g, n_g, p_g, num_of_simulations))

    initial_guess = w_initial + gamma_initial
    print(initial_guess)
    print(bounds)

    result = minimize(objective_function, initial_guess, method='Nelder-Mead', bounds=bounds)
    #result = minimize(objective_function, initial_guess, bounds=bounds)

    # The found value of w_g and gamma_g
    return result

In [36]:
%%time
Monte_carlo_params = monte_carlo_MLE(d_g_list, n_g_list, p_g, factor_loading_init, gamma_list_init,
                                     [(-1, 1), (-5, 5), (-5, 5), (-5, 5)], num_of_simulations=10000)

[0.44, -2.87, -2.3, -1.6]
[(-1, 1), (-5, 5), (-5, 5), (-5, 5)]
CPU times: total: 7.89 s
Wall time: 8.03 s


In [37]:
Monte_carlo_params

       message: Maximum number of function evaluations has been exceeded.
       success: False
        status: 1
           fun: 17.66211480811598
             x: [ 4.453e-01 -2.824e+00 -2.296e+00 -1.654e+00]
           nit: 307
          nfev: 800
 final_simplex: (array([[ 4.453e-01, -2.824e+00, -2.296e+00, -1.654e+00],
                       [ 4.453e-01, -2.824e+00, -2.296e+00, -1.654e+00],
                       ...,
                       [ 4.453e-01, -2.824e+00, -2.296e+00, -1.654e+00],
                       [ 4.453e-01, -2.824e+00, -2.296e+00, -1.654e+00]]), array([ 1.766e+01,  1.778e+01,  1.785e+01,  1.786e+01,
                        1.789e+01]))

In [38]:
Monte_carlo_params.x

array([ 0.44530394, -2.82379343, -2.29586094, -1.65412611])